In [1]:
import asyncio
from playwright.async_api import async_playwright
import statistics
import nest_asyncio

nest_asyncio.apply()

async def realizar_tarea_final():
    async def scrape_tipo(tipo):
        print(f"\n--- Extrayendo {tipo.upper()} en Huechuraba ---")
        results = []
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            url = f"https://www.portalinmobiliario.cl/venta/{tipo}/huechuraba-metropolitana"
            
            try:
                await page.goto(url, wait_until="domcontentloaded", timeout=60000)
                pagina = 1
                while True:
                    print(f"Procesando página {pagina}...")
                    await asyncio.sleep(2)
                    await page.mouse.wheel(0, 3000)
                    await asyncio.sleep(1)

                    items = await page.query_selector_all(".ui-search-result__wrapper")
                    for item in items:
                        try:
                            # 1. Obtener Precio
                            p_elem = await item.query_selector(".andes-money-amount__fraction")
                            # 2. Obtener Símbolo de moneda (UF vs $)
                            currency_elem = await item.query_selector(".andes-money-amount__currency-symbol")
                            currency = await currency_elem.inner_text() if currency_elem else ""
                            
                            card_text = await item.inner_text()
                            m2_val = None
                            if "m²" in card_text:
                                parts = card_text.split()
                                for i, part in enumerate(parts):
                                    if "m²" in part and i > 0:
                                        m2_val = float(parts[i-1].replace('.', '').replace(',', '.'))
                                        break
                            
                            if p_elem and m2_val and "UF" in currency:
                                price = float((await p_elem.inner_text()).replace('.', ''))
                                # Filtro de seguridad para evitar datos erróneos
                                if 1000 < price < 100000: 
                                    results.append({"p": price, "m": m2_val})
                        except: continue

                    # Lógica de paginación, para el navbar
                    next_btn = await page.query_selector('a[title="Siguiente"]')
                    if next_btn and await next_btn.is_visible():
                        await next_btn.click()
                        pagina += 1
                        await page.wait_for_load_state("domcontentloaded")
                    else:
                        break # Salir del bucle si no hay más páginas
                
            except Exception as e:
                print(f"Fin de navegación: {e}")
            finally:
                await browser.close()
        return results

    c_data = await scrape_tipo("casa")
    d_data = await scrape_tipo("departamento")

    def get_stats(lista):
        if not lista: return [0, 0, 0, 0]
        precios = [x['p'] for x in lista]
        uf_m2 = [x['p']/x['m'] for x in lista if x['m'] > 0]
        return [len(lista), statistics.median(precios), statistics.mean(precios), statistics.mean(uf_m2)]

    c, d = get_stats(c_data), get_stats(d_data)

    print("\n" + "="*65)
    print(f"{'Métrica':<35} | {'Casas':<12} | {'Deptos':<12}")
    print("-" * 65)
    print(f"{'Propiedades filtradas (#)':<35} | {c[0]:<12.0f} | {d[0]:<12.0f}")
    print(f"{'Mediana precio (UF)':<35} | {c[1]:<12.0f} | {d[1]:<12.0f}")
    print(f"{'Promedio precio (UF)':<35} | {c[2]:<12.0f} | {d[2]:<12.0f}")
    print(f"{'Precio por m2 (UF/m2)':<35} | {c[3]:<12.2f} | {d[3]:<12.2f}")
    print("="*65)

await realizar_tarea_final()


--- Extrayendo CASA en Huechuraba ---
Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...
Procesando página 12...
Procesando página 13...
Procesando página 14...
Procesando página 15...

--- Extrayendo DEPARTAMENTO en Huechuraba ---
Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...

Métrica                             | Casas        | Deptos      
-----------------------------------------------------------------
Propiedades filtradas (#)           | 670          | 489         
Mediana precio (UF)                 | 8570         | 5750        
Promedio precio (UF)                | 9776 